In [3]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from imageio import mimsave
%matplotlib inline

## Load Data

In [4]:
def load_img(img_path):
    img = cv2.imread(img_path)
    height, width, channel = img.shape

    return img.reshape(-1, channel), height, width

In [5]:
img1, height1, width1 = load_img("./image1.png")
img2, height2, width2 = load_img('./image2.png')

## Kernel function
<div align="center">
<img src="./img/kernel_function.png" width = "600" alt="kernel function" align=center />
</div> 

$S(x)$ is the spatial information (i.e. the coordinate of the pixel) of data $x$, and $C(x)$ is the color information (i.e. the RGB values) of data $x$. Both $\gamma_{s}$ and $\gamma_{c}$ are hyper-parameters which you can tune in your own way.

In [6]:
def kernel_function(img, width, gamma_s=0.001, gamma_c=0.001):
    # Compute S (coordinate of the pixel)
    n = len(img)
    S = np.zeros((n, 2))
    for i in range(n):
        S[i] = [i // width, i % width]

    K = squareform(np.exp(-gamma_s * pdist(S, 'sqeuclidean'))) * squareform(np.exp(-gamma_c * pdist(img, 'sqeuclidean')))

    return K

## K means

### initialize centroid

* kmeans++  
  1. Randomly pick one point as the first centroid.
  2. For each point, compute the distance from its nearest selected centroids $D(x)$.
  3. Calculate the probability of a point to be selected as the next centroid $\frac{D(x)^2}{\sum_{x \isin X}D(x)^2}$.
  4. Using Roulette wheel selection to select the next centroid.
  5. Repeat 2. ~ 4. until k centroids has been selected.   
> Roulette wheel selection: generate a radom number in interval 0 ~ 1, compare it to the cumulative probability, if it falls in the i-th interval, then the i-th point is selected as the next centroid.

In [7]:
def euclidean(point, data):
    return np.sqrt(np.sum((point - data) ** 2, axis=1))

In [8]:
def initialize_centroid(img, n_cluster=2, centroid_method='kmeans++'):
    centroids = np.zeros((n_cluster, img.shape[1]))
    if centroid_method == 'kmeans++':
        # randomly pick one point as the first centroid
        centroids[0] = img[np.random.choice(range(len(img)), size=1), :]

        for c in range(1, n_cluster):
            temp_dist = np.zeros((len(img), c))
            # for i in range(len(img)):
            for j in range(c):
                    # compute distance
                temp_dist[:, j] = euclidean(centroids[j], img)
                    #temp_dist[i, j] = np.sqrt(np.sum((img[i] - centroids[j]) ** 2))
                # for each data point, select the minimum distance (nearest centroid)
            dists = np.min(temp_dist, axis=1)
                # calulate probability
            dists /= np.sum(dists)
                # roulette wheel selection
            new_centroid_idx = np.random.choice(range(len(img)), size=1, p=dists)
            centroids[c] = img[new_centroid_idx]
                # sum = np.sum(dist) * np.random.rand()
                # for i in range(len(img)):
                #     sum -= dist[i]
                #     if sum <= 0:
                #         centroids[c] = img[i]
                #         break
    elif centroid_method == 'random':
        X_mean = np.mean(img, axis=0)
        X_std = np.std(img, axis=0)
        for c in range(img.shape[1]):
            centroids[:, c] = np.random.normal(X_mean[c], X_std[c], size=n_cluster)
    else:
        raise Exception ('unavailable centroid method!')
    
    return centroids

### defined colormap

In [9]:
np.random.seed(19990410)
colormap = np.random.choice(range(256), size=(100, 3))

### assign cluster color

In [10]:
def cluster_color(assigned_cluster, height, width, n_cluster=2):
    color = colormap[:n_cluster, :]
    res = np.zeros((len(assigned_cluster), 3), dtype=np.uint8)
    for i in range(len(res)):
        res[i, :] = color[assigned_cluster[i]]

    return res.reshape(height, width, 3)

### k means
<div align="center">
<img src="./img/k_means.png" width = "600" alt="k means" align=center />
</div>

In [11]:
def kmeans(X, height, width, n_cluster=2, centroid_method='kmeans++', EPS=1e-9):
    init_mean = initialize_centroid(X, n_cluster, centroid_method)
    assigned_cluster = np.zeros(len(X), dtype=np.uint8)
    segment = []
    n_iter = 1
    
    while True:
        # E step
        # for i in range(len(X)):
        #     euclid_dist = []
        #     for j in range(n_cluster):
        #         euclid_dist.append(np.sqrt(np.sum((X[i] - init_mean[j]) ** 2)))
        #     assigned_cluster[i] = np.argmin(euclid_dist)
        euclid_dist = np.zeros((len(X), n_cluster))
        for c in range(n_cluster):
            euclid_dist[:, c] = (euclidean(init_mean[c], X))
        assigned_cluster = np.argmin(euclid_dist, axis=1)
            

        # M step
        new_mean = np.zeros(init_mean.shape)
        for i in range(n_cluster):
            assign_i = np.argwhere(assigned_cluster == i).reshape(-1)
            for j in assign_i:
                new_mean[i] += X[j]
            if len(assign_i) > 0:
                new_mean[i] /= len(assign_i)

        diff = np.sum((new_mean - init_mean) ** 2)
        init_mean = new_mean
        assign_color = cluster_color(assigned_cluster, height, width, n_cluster)
        segment.append(assign_color)

        print(f"Iteration {n_iter}")
        for i in range(n_cluster):
            print(f"cluster={i+1} : (N pixel assigned : {np.count_nonzero(assigned_cluster == i)})")
        print(f"Difference : {diff}")
        print()
        
        n_iter += 1
        if diff < EPS:
            break
        
        
    return assigned_cluster, segment

## Kernel K means

In [12]:
W = kernel_function(img1, width=100, gamma_s=0.001, gamma_c=0.00007)
assigned_cluster, segment = kmeans(W, height=100, width=100, n_cluster=2, centroid_method='kmeans++', EPS=1e-9)

KeyboardInterrupt: 

### make gif

In [ ]:
def save_gif(img_list, gif_path="./gif", gif_name="result.gif", duration=2):
    for img in img_list:
        img = img[:, :, ::-1]
    if not os.path.exists(gif_path):
        os.makedirs(gif_path)
    mimsave(os.path.join(gif_path, gif_name), img_list, 'GIF', duration=duration)
    return

In [ ]:
save_gif(segment, "./gif", "KK_img1_k=2.gif", duration=2)

## Spectral Clustering

### Normalized Spectal Clustering
<div align="center">
<img src="./img/normalized_spectral_clustering.png" width = "800" alt="normalized spectral clustering" align=center />
</div>

In [ ]:
n_cluster=2

In [ ]:
W = kernel_function(img1, width=100, gamma_s=0.001, gamma_c=0.00007)
D = np.diag(np.sum(W, axis=1))
L = D - W
D_inverse_sqrt = np.diag(1 / np.diag(np.sqrt(D)))
L_sym = D_inverse_sqrt @ L @ D_inverse_sqrt
eigenvalue, eigenvector = np.linalg.eig(L_sym)
sort_index = np.argsort(eigenvalue)
U = eigenvector[:, sort_index[1:n_cluster+1]]
sums = np.linalg.norm(U, axis=1).reshape(-1, 1)
T = U / sums
assigned_cluster, segment = kmeans(T, height=100, width=100, n_cluster=2, centroid_method='kmeans++', EPS=1e-9)

In [ ]:
save_gif(segment, "./gif", "normalized_SC_img1_k=2.gif", duration=2)

### Unnormalized Spectral Clustering
<div align="center">
<img src="./img/unnormalized_spectral_clustering.png" width = "800" alt="unnormalized spectral clustering" align=center />
</div>

In [ ]:
W = kernel_function(img1, width=100, gamma_s=0.001, gamma_c=0.00007)
D = np.diag(np.sum(W, axis=1))
L = D - W
eigenvalue, eigenvector = np.linalg.eig(L)
sort_index = np.argsort(eigenvalue)
U = eigenvector[:, sort_index[1:n_cluster:1]]

In [ ]:
assigend_cluster, segment = kmeans(U, height=100, width=100, n_cluster=2, centroid_method='kmeans++', EPS=1e-9)

In [ ]:
save_gif(segment, "./gif", "unnormalized_SC_img1_k=2.gif", duration=2)